# Analysis. 
## Insert needed files for each PDB for running the test analysis.

The needed file is the inputs.json. We have previously created one and then, we are going to edit this each time for each PDB and export it to the PDB folder. 

This file is necessary to run this command which will run all the test analysis: 

mwf -top md.tpr -traj md.xtc -char md.tpr -pr 4 -filt -i topology --trust cohbonds

In [1]:
# Import libraries

import os 
import json
import urllib.request
from typing import List
import shutil

In [4]:
# To create the inputs.json file in the PDBs that have finished its dynamics. 


# Define an empty list
finished_run_pdbs = []

# Open the file and read the content in a list

with open('/home/imartinv/Escritorio/TFG/1.LIST_PROTEINS/Isa_dinámicas_preparacion/PDBS_finished_difference.txt', 'r') as finished_file:
    for line in finished_file:
        # Remove linebreak which is the last character of the string
        curr_place = line[:-1]
        # Add item to the list
        finished_run_pdbs.append(curr_place)


In [5]:
finished_run_pdbs

['4cof', '5zkb', '6a94', '5zk3', '5zk8', '6cm4']

In [6]:
# We define a function to obtain the description of the PDB, which will be needed for the inputs.json file 
# that is necesary for running the test analysis. 

def pdb_descr (pdb_id : str) -> List[str]:
    # Request the MMB service to retrieve pdb data
    request_url = 'https://mmb.irbbarcelona.org/api/pdb/' + pdb_id + '/entry'
    try:
        with urllib.request.urlopen(request_url) as response:
            parsed_response = json.loads(response.read().decode("utf-8"))
    # If the accession is not found in the PDB then we can stop here
    except urllib.error.HTTPError as error:
        if error.code == 404:
            return None
        else:
            raise ValueError('Something went wrong with the PDB request: ' + request_url)
    
    # Get the uniprot accessions
    
    compound = parsed_response['compound']
    
    return compound

In [7]:

for pdb_id in finished_run_pdbs: 
    # Create folder for doing the tests because this test will give us a lot of files.
    print(pdb_id)

    pdb_path = '/home/imartinv/Escritorio/ventana_a_starlife/CNS_PD_Memprot_dynamics/'+pdb_id+'_default_dppc-atomistic-simulation' # Path of the PDB folder

    directory_path_analysis = pdb_path+'/analysis_checkdynamics' # Directory we want to create. 

    if not os.path.exists(directory_path_analysis): # We create the directory if it was not previously created, just in case. 
        os.mkdir(directory_path_analysis)
        # If it was previously created, then it means that we have already created the inputs.json 
        
        # Read and modify previous inputs.json
        
        inputs_filename = '/home/imartinv/Escritorio/test/inputs.json'# Directory of where the inputs.json is (the original one)

        with open(inputs_filename, 'r') as file:
            inputs = json.load(file)
            
        # We edit parameters. 
        inputs['name'] = 'Molecular dynamics '+ pdb_descr(pdb_id)
        inputs['description'] = 'Simulation of '+ pdb_descr(pdb_id)
        inputs['links'][0]['url'] = 'http://memprotmd.bioch.ox.ac.uk/_ref/PDB/'+pdb_id+'/_sim/'+pdb_id+'_default_dppc/'
        inputs['pdbIds'] = [pdb_id]
        
        

        # To know which force field it is using
        with open('/home/imartinv/Escritorio/ventana_a_starlife/CNS_PD_Memprot_dynamics/'+pdb_id+'_default_dppc-atomistic-simulation'+'/topol.top') as topol_top:

            topol_top_lines_f = topol_top.readlines() # We read line by line the error3.log file

        for line_top in topol_top_lines_f:

            if 'gromos53a6'in line_top:
                inputs['ff']  = '53A6 GROMOS'

            elif 'charmm36' in line_top:
                inputs['ff'] = 'CHARMM36'

            elif 'gromos53a6' in line_top and 'charmm36' in line_top:
                inputs['ff'] = ['53A6 GROMOS', 'CHARMM36']


        # Export changes
        with open(inputs_filename, 'w') as file:
            json.dump(inputs, file, indent=4)
            
        
        file_json = directory_path_analysis+'/inputs.json'
        shutil.copyfile(inputs_filename,file_json)
        
        
        ## Copy other files needed for the analysis: 
        file_needed_1 = '/home/imartinv/Escritorio/ventana_a_starlife/CNS_PD_Memprot_dynamics/'+pdb_id+'_default_dppc-atomistic-simulation/100ns/md.xtc'

        file_needed_2 = '/home/imartinv/Escritorio/ventana_a_starlife/CNS_PD_Memprot_dynamics/'+pdb_id+'_default_dppc-atomistic-simulation/100ns/md.tpr'

        shutil.copyfile(file_needed_1,directory_path_analysis+'/md.xtc')
        shutil.copyfile(file_needed_2,directory_path_analysis+'/md.tpr')



4cof
5zkb
6a94
5zk3
5zk8
6cm4
